In [1]:
# import library
from HZ_API import tokenAPI, actuatorAPI, sensorAPI
from HZ_API import controlAPI, crossNVAPI, south_window, roof_shading,  valve, valve31,valve32,valve33
from HZ_API import outdoorT, windspeed, winddirection, z31roomT, z31CO2, z31RH, z31slabT, z31occ
from HZ_API import z31south, z31roof, z31shading, z31valve
from HZ_API import EnergyRate, z31supplyT, z31returnT, z31vol

from HZ_NOAAv5 import ClientNOAA


In [2]:
import pytz
import datetime
import cvxpy as cp
import pandas as pd
import numpy as np
import pickle
import time
import csv

from statistics import mean 
from datetime import datetime,timedelta

### Natural Ventilation

In [3]:
# update Nov 19
b_CO2 = np.array([10.09, -55.33 , -2.558, 0, 0, -0.3529])
a_CO2 = 1.002

#Winter 2020 upated Nov 19
b_T = np.array([0, -0.191, 0, 0.1609, 0.119, -0.1227])
a_T = 1.004

def winterNV_MPC(token, Temps_low, Temps_high, CO2_goal, sch):
    
    # use only 4 for winter MPC NV
    Out_T = outdoorT(token)
    Wind_direction = winddirection(token)
    Wind_speed = windspeed(token)
    Room_T = z31roomT(token) - 1.5
    Room_CO2 = z31CO2(token) + 100
    z31s = z31south(token)
    occ = z31occ(token)
    # wall mounted sensor has a systematic error of +1.5°c and - 100 ppm
    
    print("----------------------------")
    nowtime = datetime.now()
    current_time = nowtime.strftime("%B %d, %Y %H:%M:%S")
    print("Current Time =", current_time)
    
    print("outdoor temp is", Out_T)
    print("wind speed is", Wind_speed)
    print("wind direction is", Wind_direction)
    print("South windiow's position is at", z31s)
    print("Occupancy is", occ)
    
    # schedual 
    #sch = [1,0,1,0,1,1,1,1,1,1,1]
    # prepare the features (coef of control action U)
    Del_T = np.abs(Room_T - Out_T)
    S_Del_T = np.sqrt(Del_T)

    In_Angle = 191.55 - Wind_direction
    SinV = Wind_speed*np.abs(np.sin(In_Angle*np.pi/180))
    CosV = Wind_speed*np.cos(In_Angle*np.pi/180)
    A_CosV = np.abs(CosV)
    
    # prepare the Matrix for A and B
    data_CO2 = np.array([Del_T, S_Del_T, SinV, CosV, A_CosV, Wind_speed])
    Bm_CO2 = np.inner(b_CO2, data_CO2)
    print("CO2 Matrix",Bm_CO2)

    data_T = np.array([Del_T, S_Del_T, SinV, CosV, A_CosV, Wind_speed])
    Am_T = np.inner(b_T, data_T)
    print("Temps Matrix", Am_T)
    
    # MPC model with CVXPY
    p = occ # person
    G = 55*p # CO2 generation for 5 mins

    n = 2  #number of states
    m = 1  #number of action
    T = 11 #time horizon, 1 means 5 mins: 150 min
    x = cp.Variable((n, T+1))
    u = cp.Variable((m,T), nonneg=True)
    alpha = cp.Variable((1,T),nonneg=True) # offset to high CO2
    beta = cp.Variable((1,T),nonneg=True) # offset to low CO2
    ro = cp.Variable((1,T),nonneg=True) # offset for low temps
    ro1 = cp.Variable((1,T),nonneg=True) # offset for low temps
    
    # for change of indoor temp
    #d_ro = cp.Variable((1,T),nonneg=True)
    
    # A, B can be expressed with cp.Parameter, ideal for LTV system with NOAA
    A = np.matrix([[a_T, 0],[0, a_CO2]])
    B = np.matrix([[Am_T],[Bm_CO2]])
    C = np.matrix([[0],[G]])
    C1 = np.matrix([[0],[55]])
    
    # MPC problem formulation 
    cost = 0
    constr = []

    #T_0 = 26.5
    # sensor has a systematic error of +1.5°c
    T_0 = Room_T
    print("Room Temps now is", T_0)
    # temperature determine the opening largely offset 100 ppm
    
    #Test use
    #CO2_0 = 850
    CO2_0 = Room_CO2
    print("CO2 now is", CO2_0)
    for t in range(T-1):
        cost += cp.sum_squares(u[:,t] - u[:, t-1])+10*cp.sum_squares(u[:,t])+\
        cp.sum_squares(alpha[:,t]+20*beta[:,t]+40*ro[:,t]+20*ro1[:,t])
        # high temp limit coef from 40 to 20
        constr += [x[:,t+2] == A@x[:,t+1] + B@u[:,t+1] + cp.reshape(C1,(2,))*sch[t+1],
                x[0,t-1] - x[0,t] <= 1,
                Temps_low - ro[:,t]<= x[0,t],
                x[0,t]<= Temps_high + ro1[:,t],
                x[1,t]<= CO2_goal + alpha[:,t],
                800 - beta[:,t] <= x [1,t],
                u[:,t] <= 1]
        # update 400 ppm to silence beta punishment
        #constr += [d_ro[:,t]== (Out_T - x[0,t])*(x[1,t+1]-x[1,t])/(x[1,t]-400)]
    # dynamic 
    # adjust the constrain xt - x(t-1) <1 for summer 
    # 800 ppm is the ideal control objective
    constr += [x[0,0] == T_0, x[1,0]== CO2_0, x[:,1] == A@x[:,0] + B@u[:,0] + cp.reshape(C,(2,)), u[:,T-1] <= 1]
    # occupancy sensor works here


    prob = cp.Problem(cp.Minimize(cost), constr)
    prob.solve(solver = cp.GUROBI, verbose=False, warm_start = True)
    
    if prob.status == 'infeasible_inaccurate': 
        print("optimization problem is",prob.status)
        position_to_set = 0
        place_hold = [0]
        place_hold1 = [Room_T] 
        x_place = [ele for ele in place_hold1 for i in range(T)]
        ro1_place = [ele for ele in place_hold for i in range(T)]
        
        return x_place, position_to_set, ro1_place
    
    else:
        print("NV MPC return a position:",u.value[0])
        position_to_set = np.rint(u.value[0][0]*100)
    
        print(x.value)
        print("window opening will be at",position_to_set)

        return x.value[0], position_to_set, ro1.value[0]


### TABS

In [9]:
def TABS_estimation(token):
    lisBTU = []
    t0 = time.time()
    while (time.time() < t0 + 5*60):
        v_position = z31valve(token)
        print("valve position:",v_position)
        if v_position>10:
            lisBTU.append(EnergyRate(token))
            time.sleep(60)
        else:
            controlAPI(token, valve, 100)
            lisBTU.append(EnergyRate(token))
            time.sleep(5)
    print(lisBTU)        
    #listBTU = [i for i in lisBTU if i > 0.0]
    
    return max(mean(lisBTU),1.0)
        
        
def TABS_NV_action(token, position_to_set_v, position_to_set_w):
    t0 = time.time()
    while (time.time() < t0 + 5*60):
        #for 15 min
        controlAPI(token, valve, position_to_set_v)
        #for 5 min
        controlAPI(token, south_window, position_to_set_w)

def TABS_preoccupancy(token, Temps_high, Temps_low, weather_case, TABS, T_seq):

    solar_E = weather_case.solarI_nowII()
    solar_E_list = solar_E['solar_radiation'].to_list()
    print(solar_E)

    #token = tokenAPI()
    #Temps_high = 22
    print("----------------------------")
    nowtime = datetime.now()
    current_time = nowtime.strftime("%B %d, %Y %H:%M:%S")
    print("Current Time =", current_time)

    T = 3 #time horizon: 10*15

    m,n = 1,1
    # slab surface
    x = cp.Variable((m, T+1))
    u = cp.Variable((n, T), boolean = True)

    alpha = cp.Variable((1,T),nonneg=True) # offset to hight slab surface temperature
    beta = cp.Variable((1,T),nonneg=True)
    # room air temps will be a sequence 
    T_room_air = T_seq[0::3]
    #T_room_air = z31roomT(token)-1.5
    
    T_slab_c = z31slabT(token)
    
    T_0 = 0.598*(T_room_air[0])+0.707*T_slab_c-4.37-1
    #TABS = 0.9
    print("Room air temperature:", T_room_air[0])
    print("Slab core temperature:", T_slab_c)
    print("Estimated slab surface temperature:", T_0)

    cost = 0
    constr = []

    ####
    #MPC
    ###
    for t in range(T):
    
        cost += cp.sum_squares(u[:,t]) + cp.sum_squares(u[:,t] - u[:, t-1])+cp.sum_squares(u[:,t]) + 200*cp.sum_squares(alpha[:,t])
        constr += [x[:,t+1] == 0.973*x[:,t] + 0.000482*0.929*TABS*u[:,t] +\
                   0.000482*0.263*solar_E_list[t]-0.000482*30 + 0.027*T_room_air[t], 
                   x[:,t]<= Temps_high + alpha[:,t],Temps_low - beta[:,t]<=x[:,t]]
    
    constr += [x[0,0] == T_0]
    
    prob = cp.Problem(cp.Minimize(cost), constr)
    prob.solve(solver = cp.GUROBI, verbose=False, warm_start = True)
    
    if prob.status == 'infeasible_inaccurate': 
        print("optimization problem is",prob.status)
        position_to_set = 0
    
    else:
        
        print("TABS MPC return a position:",u.value[0])
        position_to_set = np.rint(u.value[0][0]*100)  
        #TABS_action(token, position_to_set)
    
    print(x.value)
    print("valve will be at",position_to_set)
    return position_to_set
    # update the TABS energy rate


### room overheated controller

In [5]:
def feedback_layer(token, T_seq, ro1_seq):
    outT = outdoorT(token)
    k = 0.15*10
    
    return (T_seq[0]-outT)*ro1_seq[0]*k

### Coupling 

In [6]:
def TABS_NV_winter(token, Temps_high_slab, Temps_low_slab, case, \
                   TABS_rate, Temps_low, Temps_high, CO2_goal, sch):
    
    
    acc = 3
    valve_pos = 0
    
    for k in range(acc):
        # NV
        T_seq, south_pos_0, ro1_seq = winterNV_MPC(token, Temps_low, Temps_high, CO2_goal,sch)
    
        south_pos_1 = feedback_layer(token, T_seq, ro1_seq)
        print("feed back layer:",south_pos_1)
        # TABS
    
        if k%3 == 0:
            TABS_power = 293*TABS_rate
            valve_pos = TABS_preoccupancy(token, Temps_high_slab, Temps_low_slab, case, TABS_power, T_seq)
    
        # coupling
        south_pos = max(south_pos_0, south_pos_1)
    
        TABS_NV_action(token, valve_pos, south_pos)
    
    if z31valve(token)>10:
        TABS_new = TABS_rate*0.9 + EnergyRate(token)*0.1 
        TABS_rate = TABS_new
    
    return TABS_rate
    
    
    

### Main function

In [10]:
case = ClientNOAA()
info_DF = case.weather_DF()
print(info_DF.head())

Now is one hour ago 2021-02-25 08:49:19.747578-05:00
Weather Forcast is updated by NOAA at: 2021-02-25T11:29:58+00:00
--------------------------------------------
                   date_time temperature windspeed direction sky_condition  \
0  2021-02-25 09:00:00-05:00        6.11      0.45       315         Sunny   
1  2021-02-25 10:00:00-05:00        6.67      0.45       315         Sunny   
2  2021-02-25 11:00:00-05:00        6.67      0.45       315         Sunny   
3  2021-02-25 12:00:00-05:00        6.67      0.45       315         Sunny   
4  2021-02-25 13:00:00-05:00        6.67      0.45       270         Sunny   

  daytime  zenith  azimuth  ClearSkySolar  Cloud  CloudSolar  SolarRef  
0       1   65.83   130.73          375.2    0.0       375.2     841.5  
1       1   58.39   145.23          507.7    0.0       507.7     908.7  
2       1   53.44   162.10          590.8    0.0       590.8     939.3  
3       1   51.76   180.64          618.1    0.0       618.1     947.9  
4  

In [ ]:
# update the weather info
# this function can be used to update info in the loop
case.weather_DF()

In [11]:
token = tokenAPI()
t0 = time.time()
H = 6
#valve_state = [1]
TABS_rate = TABS_estimation(token)
#TABS_rate = 1.0
print("BTU Energy rate reading:",TABS_rate)

# next H+4 hourly schedule to 5 min level 
sch_H = [1,1,1,1,1,0,0,0,0,0]
K = 12

sch_5min = [ele for ele in sch_H for i in range(K)]

while (time.time() < t0 + H*3600):
    
    # air 
    Temps_low = 20
    Temps_high = 24
    CO2_goal = 1000
    # slab surface
    Temps_high_slab = 25
    
    Temps_low_slab = 22
    

    TABS_rate = TABS_NV_winter(token, Temps_high_slab, Temps_low_slab, case, \
                               TABS_rate, Temps_low, Temps_high, CO2_goal, sch_5min)
    
    
    if len(sch_5min)>2*K:
        sch_5min.pop(0)


Token received.
valve position: 12.0
valve position: 12.0
valve position: 13.0
valve position: 13.0
valve position: 12.0
[0.0, 0.0, 0.0, 0.0, 0.0]
BTU Energy rate reading: 1.0
----------------------------
Current Time = February 25, 2021 09:54:34
outdoor temp is 8.27000045776367
wind speed is 4.90000009536743
wind direction is 326.0
South windiow's position is at 0.0
Occupancy is 1
CO2 Matrix -72.69419569761939
Temps Matrix -1.4923505432347774
Room Temps now is 23.5799999237061
CO2 now is 685.599975585938
Academic license - for non-commercial use only - expires 2021-04-19
Using license file C:\Users\wez664\gurobi.lic
NV MPC return a position: [5.74936056e-17 2.21739123e-15 3.63634855e-01 3.93425239e-01
 3.95199527e-01 3.94609325e-01 3.93645540e-01 3.90547512e-01
 3.62196407e-01 3.29269464e-02 4.95760047e-05]
[[  23.57999992   23.67431992   23.7690172    23.3214226    22.82757992
    22.32911401   21.82953503   21.32939603   20.83187982   20.37468334
    20.40704353   20.48859772]
 [ 68

Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator

Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  33.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuato

feed back layer: 0.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  58.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  7.0
Absolute position south
The actuator South Window is opening, at:  14.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  58.0
Absolute position south
The actuator South Window is opening, at:  47.0
Absolute position south
The actuator South Window is opening, at:  55.0
Absolute 

Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuator South Window is opening, at:  0.0
Absolute position south
The actuato

...

Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  26.878825179290942
Absolute position south
The actuator South Window is opening, at:  28.0
Absolute position south
The actuator South Window is opening, at:  27.0
Absolute position south
The actuator South Window is opening, at:  27.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  26.878825179290942
Absolute position south
The actuator South Window is opening, at:  27.0
Absolute position south
The actuator South Window is opening, at:  27.0
Absolute position south
The actuator South Window is opening

Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  25.22518729190792
Absolute position south
The actuator South Window is opening, at:  25.0
Absolute position south
The actuator South Window is opening, at:  25.0
Absolute position south
The actuator South Window is opening, at:  25.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  25.22518729190792
Absolute position south
The actuator South Window is opening, at:  25.0
Absolute position south
The actuator South Window is opening, at:  25.0
Absolute position south
The actuator South Window is opening, at:  25.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, a

Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  28.009797959519705
Absolute position south
The actuator South Window is opening, at:  28.0
Absolute position south
The actuator South Window is opening, at:  28.0
Absolute position south
The actuator South Window is opening, at:  28.0
----------------------------
Current Time = February 25, 2021 15:39:20
outdoor temp is 7.67999982833862
wind speed is 4.0
wind direction is 8.0
South windiow's position is at 28.0
Occupancy is 1
CO2 Matrix -57.00165533259163
Temps Matrix -1.45663201756662
Room Temps now is 25.1599998474121
CO2 now is 490.079986572266
NV MPC return a position: [3.35024478e-18 5.21313688e-18 9.20451334e-18 2.04313933e-17
 7.85558651e-17 4.84983045e-01 7.55340993e-01 1.16359844e-

South Window
Command is sucessfully sent to actuator:  32.37268013935201
Absolute position south
The actuator South Window is opening, at:  31.0
Absolute position south
The actuator South Window is opening, at:  33.0
Absolute position south
The actuator South Window is opening, at:  33.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  32.37268013935201
Absolute position south
The actuator South Window is opening, at:  33.0
Absolute position south
The actuator South Window is opening, at:  33.0
Absolute position south
The actuator South Window is opening, at:  33.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, a

Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  30.28408303584983
Absolute position south
The actuator South Window is opening, at:  30.0
Absolute position south
The actuator South Window is opening, at:  30.0
Absolute position south
The actuator South Window is opening, at:  30.0
Heating valve
Command is sucessfully sent to actuator:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
Heating valve
The actuator Heating valve is opening, at:  0.0
South Window
Command is sucessfully sent to actuator:  30.28408303584983
Absolute position south
The actuator South Window is opening, at:  30.0
Absolute position south
The actuator South Window is opening, at:  30.0
Absolute position south
The actuator South Window is opening, 